In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import sys, os
# This is not super pretty, but I think this is the best way to import stuff from ../../../util?
CODE_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
if CODE_ROOT not in sys.path:
    sys.path.insert(1, CODE_ROOT)

from util.spectra_helpers import SpectraCustomHDF5

In [ ]:
suite_to_use = "L25n256_suite"
index_list = [i for i in range(4)]
spec_num = 512

fluxes = []
for i in index_list:
    path_to_file = f"/vera/ptmp/gc/jerbo/training_data/{suite_to_use}_snr_10/gp{i}_spectra.hdf5"
    spec_file = SpectraCustomHDF5(path_to_file)
    wavelengths, flux = spec_file.get_single_spectrum(spec_num)
    fluxes.append(flux)


In [ ]:
fig, ax = plt.subplots(figsize=(13, 10))

ax.set_xticks([])
ax.set_yticks([])
for spine in ax.spines.values():
    spine.set_visible(False)

gs = gridspec.GridSpec(len(fluxes), 1, hspace=0)
axes = []

for i in range(len(fluxes)):
    ax = fig.add_subplot(gs[i, 0])

    ax.plot(wavelengths, fluxes[i], label="simulated spectra")
    ax.annotate(f"Box {index_list[i]}", (wavelengths[0], min(fluxes[i])))
    # ax.plot(wavelengths, patched_spectra[i], color="black", label="original spectrum")
    ax.set_xlim([3550, 3950])
    ax.set_ylabel("Relative Flux")
    # ax.legend(loc="upper left")
    ax.tick_params(axis='x', direction='in')
    if i == len(fluxes)-1:
        ax.set_xlabel(r"Wavelength [$\AA$]")
    if i < len(fluxes)-1:
        ax.tick_params(labelbottom=False)

    axes.append(ax)

# fig.supxlabel(r"Wavelength [$\AA$]")
# fig.supylabel("Relative Flux")

# plt.savefig("plots/noise_spectra.pdf", format="PDF")
plt.show()

In [ ]:
from astropy.table import Table

lya_cat = Table.read("data/BOSSLyaDR9_cat.fits")
pmf_list = []
# take first 10
count = 0
for row in lya_cat:
    if row["SNR"] > 10:
        # print(f"({row["PLATE"]}, {row["MJD"]}, {row["FIBERID"]}), ")
        pmf_list.append((row["PLATE"], row["MJD"], row["FIBERID"]))

        count +=1
        #if count > 30:
        #    break
print(count)

In [ ]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

# (plate, mjd, fiber)
"""pmf_list = [
    (4296, 55499, 630), 
    (4217, 55478, 519), 
    (4216, 55477, 166), 
    (4296, 55499, 756), 
    (4365, 55539, 204), 
    (4217, 55478, 933), 
    (4217, 55478, 96), 
    (4218, 55479, 394), 
    (4218, 55479, 696), 
    (4218, 55479, 852), 
    (4219, 55480, 222),
]"""

wavelengths_boss_specs = []
fluxes_boss_specs = []

for plate, mjd, fiber in pmf_list:
    fiber_str = f"{fiber:04d}"
    filename = f"speclya-{plate}-{mjd}-{fiber_str}.fits"
    file_path = f"/virgotng/mpia/obs/SDSS/BOSSLyaDR9_spectra/{plate}/" + filename

    with fits.open(file_path) as hdul:
        data = hdul[1].data

        loglam   = data["LOGLAM"]
        flux     = data["FLUX"]
        ivar     = data["IVAR"]
        cont     = data["CONT"]
        dla_corr = data["DLA_CORR"]
        mask_comb = data["MASK_COMB"]   # <-- IMPORTANT

    # --- Read RESID ---
    resid_file = "data/residcorr_v5_4_45.dat"
    resid = []
    resid_lam = []

    with open(resid_file) as rfile:
        reader = csv.reader(rfile, delimiter=" ")
        for i, row in enumerate(reader):
            if i != 0:
                resid.append(float(row[-1]))
                resid_lam.append(float(row[1]))

    resid_lam = np.array(resid_lam)
    resid = np.array(resid)

    i0 = np.where(np.isclose(resid_lam, loglam[0]))[0][0]
    i1 = np.where(np.isclose(resid_lam, loglam[-1]))[0][0] + 1
    resid = resid[i0:i1]

    # Convert to angstroms
    wavelength = 10**loglam

    # ---------------------------------------------------
    # SDSS-recommended pixel mask
    # ---------------------------------------------------
    good_pixel = (
        (mask_comb == 0) &     # SDSS quality mask
        (ivar > 0)
    )

    # Initialize F with NaNs
    F = np.full_like(flux, np.nan)

    # Compute transmission ONLY for good pixels
    F[good_pixel] = (flux[good_pixel] * dla_corr[good_pixel] / ((cont[good_pixel] * resid[good_pixel])))
    # F = (flux * dla_corr / ((cont * resid)))

    # --- Select Lyα forest region ---
    forest_mask = (
        (wavelength >= 3600.0) &
        (wavelength <= 3950.0)
    )

    wave_sel = wavelength[forest_mask]
    F_sel    = F[forest_mask]

    # print(len(F_sel), len(wave_sel))

    if not np.any(np.isfinite(F_sel[:20])):
        continue   # skip this spectrum

    wavelengths_boss_specs.append(wave_sel)
    fluxes_boss_specs.append(F_sel)


In [ ]:
print(len(fluxes_boss_specs)) # 2293

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

fluxes_boss_specs_plot = fluxes_boss_specs[:4]
wavelengths_boss_specs_plot = wavelengths_boss_specs[:4]

fig, ax = plt.subplots(figsize=(13, 10))

ax.set_xticks([])
ax.set_yticks([])
for spine in ax.spines.values():
    spine.set_visible(False)

gs = gridspec.GridSpec(len(fluxes_boss_specs_plot), 1, hspace=0)
axes = []

for i in range(len(fluxes_boss_specs_plot)):
    ax = fig.add_subplot(gs[i, 0])
    # print(fluxes_boss_specs_plot[i])
    ax.plot(wavelengths_boss_specs_plot[i], fluxes_boss_specs_plot[i], label="simulated spectra")
    # ax.annotate(f"Box {index_list[i]}", (wavelengths_boss_specs_plot[0], min(fluxes_boss_specs_plot[i])))
    # ax.plot(wavelengths, patched_spectra[i], color="black", label="original spectrum")
    # ax.set_xlim([left_cut, right_cut])
    ax.set_ylabel("Relative Flux")
    ax.set_xlim([3600, 3950])
    # ax.legend(loc="upper left")
    ax.tick_params(axis='x', direction='in')
    if i == len(fluxes_boss_specs_plot)-1:
        ax.set_xlabel(r"Wavelength [$\AA$]")
    if i < len(fluxes_boss_specs_plot)-1:
        ax.tick_params(labelbottom=False)

    axes.append(ax)

# fig.supxlabel(r"Wavelength [$\AA$]")
# fig.supylabel("Relative Flux")

# plt.savefig("plots/noise_spectra.pdf", format="PDF")
plt.show()

In [ ]:
import torch

tensor_specs = []
for spec in fluxes_boss_specs:
    if len(spec) == 402:
        tensor_specs.append(spec)

t = torch.Tensor(np.array(tensor_specs))

print(t.shape)

In [ ]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

file_path = f"/virgotng/mpia/obs/SDSS/Delta_LYA/delta-432.fits"

with fits.open(file_path) as hdul:
    data = hdul[1].data

    loglam   = data["LOGLAM"]
    flux     = data["DELTA"]
    weight     = data["WEIGHT"]
    cont     = data["CONT"]

# Convert to angstroms
wavelength = 10**loglam

# --- Select Lyα forest region ---
forest_mask = (
    (wavelength >= 3550.0) &
    (wavelength <= 3950.0)
)

wave_sel = wavelength[forest_mask]
F_sel    = flux[forest_mask] * cont[forest_mask]

plt.plot(wave_sel, F_sel)
plt.show()
